In [73]:
pip install scikit-surprise

47774.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [69]:
df_final = pd.read_csv(f"{data_path}/final_svd_dataset.csv")

In [ ]:
df_final['rating'].describe()

count    5.970537e+06
mean     3.919667e+00
std      9.910473e-01
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [72]:
df_final.columns

Index(['user_id', 'book_id', 'rating', 'decade'], dtype='object')

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy


# Define reader for Surprise (rating scale 1 to 5)
reader = Reader(rating_scale=(1, 5))

# Load data into Surprise format
data = Dataset.load_from_df(df_final[['user_id', 'book_id', 'rating']], reader)

# Split into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Apply SVD
svd = SVD(n_factors=100, random_state=42)
svd.fit(trainset)

# Predict on test set (optional evaluation)
predictions = svd.test(testset)
rmse = accuracy.rmse(predictions)

# ---------------------
# Generate 25 recommendations per user
# ---------------------

# Get list of all users and books
all_user_ids = df_final['user_id'].unique()
all_book_ids = df_final['book_id'].unique()

# Build recommendation list
user_recommendations = {}

for user_id in all_user_ids:
    # Get books the user has already rated
    rated_books = df_final[df_final['user_id'] == user_id]['book_id'].tolist()
    
    # Predict ratings for all unseen books
    unseen_books = [book for book in all_book_ids if book not in rated_books]
    
    predictions = [(book, svd.predict(user_id, book).est) for book in unseen_books]
    
    # Sort by predicted rating
    top_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:25]
    
    recommended_books = [book for book, _ in top_predictions]
    user_recommendations[user_id] = recommended_books

# Example output for one user
example_user = list(user_recommendations.keys())[0]
print(f"Top 25 recommendations for user {example_user}: {user_recommendations[example_user]}")
